In [28]:
import yfinance as yf
import pandas as pd
import mysql.connector as connector
import datetime

In [ ]:
#!pip install mysql-connector-python

## Importing data using yfinance

In [2]:
stock = yf.Ticker("NVDA") 

In [3]:
stock_historical = stock.history(period="max", interval="1d")
stock_historical

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1999-01-22 00:00:00-05:00,0.040114,0.044770,0.035577,0.037607,2714688000,0.0,0.0
1999-01-25 00:00:00-05:00,0.040591,0.042024,0.037607,0.041547,510480000,0.0,0.0
1999-01-26 00:00:00-05:00,0.042024,0.042860,0.037726,0.038323,343200000,0.0,0.0
1999-01-27 00:00:00-05:00,0.038442,0.039398,0.036293,0.038204,244368000,0.0,0.0
1999-01-28 00:00:00-05:00,0.038204,0.038442,0.037845,0.038084,227520000,0.0,0.0
...,...,...,...,...,...,...,...
2025-11-17 00:00:00-05:00,185.970001,189.000000,184.320007,186.600006,173628900,0.0,0.0
2025-11-18 00:00:00-05:00,183.380005,184.800003,179.649994,181.360001,213598900,0.0,0.0
2025-11-19 00:00:00-05:00,184.789993,187.860001,182.830002,186.520004,247246400,0.0,0.0


In [4]:
df = pd.DataFrame(stock_historical)

## SQL Connection

In [5]:
try:
    conn = connector.connect(user='root', password='123456', host='localhost', port='3306', database='my_db')
except connector.Error as er:
    print('Error code:', er.errorno)
    print('Error msg:', er.msg)

In [6]:
mycursor = conn.cursor()

mycursor.execute("SELECT * FROM stock_price")

results =  mycursor.fetchall()


In [25]:
last = results[-1][0]

In [26]:
last

datetime.date(2025, 11, 21)

In [29]:
today = datetime.date.today()

In [ ]:
if last < today:
    for index, row in df.iterrows():
            sql = "INSERT INTO stock_price (date, open, high, low, close, volume) VALUES (%s, %s, %s, %s, %s, %s)"
            val = (index.date(), row['Open'], row['High'], row['Low'], row['Close'], row['Volume'])
            mycursor.execute(sql, val)
            conn.commit()
            print(mycursor.rowcount, "record inserted.")

## Inserting data into sql table stock_price

In [ ]:
#sql ="INSERT INTO"

sql = "INSERT IGNORE INTO stock_price (Date, Open, High, Low, Close, Volume, Dividends, Stock_Splits) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"

# Convert DataFrame to list of tuples without the index
tuples_list = list(df.itertuples(index=True, name=None))

#need to use executemany to insert multiple rows
mycursor.executemany(sql, tuples_list)

In [31]:
#commit from the connection
conn.commit()

mycursor.close()
conn.close()